In [2]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False

  def preprocessing(self, sequences):
    result = []

    import re
    """
    result = [re.sub('[^a-zA-Z0-9]',' ', a_sequence).lower().split() for a_sequence in sequences]
    """
    #조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
    #조건 2: 토큰화는 white space 단위로 수행합니다.
    #output: 각 문장을 토큰화한 결과로, nested list 형태입니다.
    for a_sequence in sequences:
      result.append(re.sub('[^a-zA-Z0-9]',' ', a_sequence).lower().split())

    return result

  def fit(self, sequences):
    self.fit_checker = False

    index = 1
    #조건 1: 위에서 만든 preprocessing 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
    for token in self.preprocessing(sequences):
      for word in token:
        if word not in self.word_dict:
          #조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
          self.word_dict[word] = index
          index += 1
          
    self.fit_checker = True
  
  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:

      for token in tokens:
        #조건 1: 어휘 사전(self.word_dict)에 없는 단어는 'oov'의 index로 변환합니다.
        #output: 각 문장의 정수 인덱싱으로, nested list 형태입니다.
        result.append([self.word_dict[vocab] if vocab in self.word_dict else self.word_dict['oov'] for vocab in token])

      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [5]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)

    self.idf = []
    import collections
    import numpy as np
    d_sequences = {'bow_{}'.format(i):list(set(b)) for i,b in enumerate(tokenized)}
    #n : 입력된 전체 문장 개수
    n=len(d_sequences.keys())
    l_sequences = sum(d_sequences.values(),[])
    #df(d,t) : 단어 t가 포함된 문장 d의 개수
    df = dict(collections.Counter(l_sequences))
    idf_tokens = dict.fromkeys(set(l_sequences), 0)
    for word in set(l_sequences):
      idf_tokens[word] = np.log(n/(1+df[word]))
    #조건 1: IDF 행렬은 list 형태입니다.
    self.idf.extend(list(idf_tokens.values()))

    self.fit_checker = True
    
  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)

      self.tfidf_matrix = []
      import collections
      import numpy as np
      tf_matrix=[]
      for token in tokenized:
        tf_dic = dict.fromkeys(set(sum(tokenized, [])),0)
        tf_counter = dict(collections.Counter(token))
        for word in token:
          tf_dic[word]=tf_counter[word]
        #조건1 : 입력 문장을 이용해 TF 행렬(tf_matrix)을 만드세요.
        #tf(d, t) : 문장 d에 단어 t가 나타난 횟수
        tf_matrix.append(list(tf_dic.values()))
      result = np.reshape(tf_matrix, (len(tokenized),-1))

      #조건2 : 문제 2-1( fit())에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
      tf_idf = result * self.idf

      #output : nested list 형태입니다.
      self.tfidf_matrix.extend(tf_idf.tolist())

      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")
  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)